In [64]:
import pandas as pd
import talib
import random

In [108]:
data_df = pd.read_csv("data/ETHUSD-5m-data.csv")



In [109]:
def convert_data_frame_5m_to_30m(data):
    converted_data = []
    if data.shape[0] > 6:
        for i in range(0, data.shape[0] - 5, 6):
            high = data[i:i + 5]['high'].max()
            low = data[i:i + 5]['low'].min()
            volume = data[i:i + 5]['volume'].sum()
            converted_data.append({
                'timestamp': data.iloc[i]['timestamp'],
                'open': data.iloc[i]['open'],
                'high': high,
                'low': low,
                'close': data.iloc[i + 5]['close'],
                'volume': volume
            })
    else:
        raise Exception('Too few 5m candles')
    return pd.DataFrame(converted_data)


In [110]:
data_df = convert_data_frame_5m_to_30m(data_df)

In [111]:
data_df

,timestamp,open,high,low,close,volume
0,2018-08-02 09:05:00+00:00,NaN,435.50,400.00,422.20,9566
1,2018-08-02 09:35:00+00:00,422.20,422.50,417.15,415.40,59126
2,2018-08-02 10:05:00+00:00,415.40,427.00,410.00,413.05,324576
3,2018-08-02 10:35:00+00:00,413.05,416.95,412.80,417.00,87358
4,2018-08-02 11:05:00+00:00,417.00,417.00,415.00,417.70,49692
...,...,...,...,...,...,...
40471,2020-11-22 12:35:00+00:00,528.35,530.45,526.50,527.40,1081717
40472,2020-11-22 13:05:00+00:00,527.40,528.75,525.30,531.90,1015441
40473,2020-11-22 13:35:00+00:00,531.90,534.60,531.00,533.20,1192404
40474,2020-11-22 14:05:00+00:00,533.20,535.50,529.80,532.95,1001709


In [121]:
params = {
    'RSI_BULL': 10,
    'RSI_BULL_TF': 19,
    'RSI_BULL_HIGH': 81.93600697495238,
    'RSI_BULL_LOW' : 45.516632226052714,
    'RSI_BEAR': 15,
    'RSI_BEAR_TF': 16,
    'RSI_BEAR_HIGH': 45.61985405602511,
    'RSI_BEAR_LOW' : 20.925647327774335,
    'SMA_FAST': 138,
    'SML_SLOW': 67,
    'SMA_TF': 6,
    'ADX': 3,
    'ADX_TF': 9,
    'ADX_HIGH': 65.80525318969225,
    'ADX_LOW': 45.6513373850286,
    'BULL_MOD_HIGH': 5,
    'BULL_MOD_LOW' : -5,
    'BEAR_MOD_HIGH': 15,
    'BEAR_MOD_LOW' : -5,
    'STOP_LOSS': 20, # %,
    'TAKE_PROFIT': 14 # %
}

params2 = {'RSI_BULL': 14, 'RSI_BULL_TF': 6, 'RSI_BULL_HIGH': 70.66602096693327, 'RSI_BULL_LOW': 47.72774022537839, 'RSI_BEAR': 19, 'RSI_BEAR_TF': 20, 'RSI_BEAR_HIGH': 47.831780740039356, 'RSI_BEAR_LOW': 19.675569028578884, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 9, 'ADX': 15, 'ADX_TF': 5, 'ADX_HIGH': 62.76900764282739, 'ADX_LOW': 30.616489167985236, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 12.475810949883751, 'TAKE_PROFIT': 10.09271064841754}
params3 = {'RSI_BULL': 16, 'RSI_BULL_TF': 20, 'RSI_BULL_HIGH': 71.15720121829733, 'RSI_BULL_LOW': 47.5028232805205, 'RSI_BEAR': 18, 'RSI_BEAR_TF': 11, 'RSI_BEAR_HIGH': 38.73422848226052, 'RSI_BEAR_LOW': 20.191061873397032, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 8, 'ADX': 6, 'ADX_TF': 16, 'ADX_HIGH': 90.73868788251929, 'ADX_LOW': 25.35677072649313, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 13.250081908012456, 'TAKE_PROFIT': 16.4993153742895}
params4 = {'RSI_BULL': 18, 'RSI_BULL_TF': 8, 'RSI_BULL_HIGH': 65.61862224052749, 'RSI_BULL_LOW': 43.02113912133937, 'RSI_BEAR': 15, 'RSI_BEAR_TF': 8, 'RSI_BEAR_HIGH': 58.39183822274852, 'RSI_BEAR_LOW': 12.656621672290465, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 5, 'ADX': 5, 'ADX_TF': 16, 'ADX_HIGH': 77.57150575486634, 'ADX_LOW': 56.41056770151446, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 0.8330569920731217, 'TAKE_PROFIT': 31.093215318561775}
params5 = {'RSI_BULL': 18, 'RSI_BULL_TF': 14, 'RSI_BULL_HIGH': 70.98853186912616, 'RSI_BULL_LOW': 46.045622334549414, 'RSI_BEAR': 16, 'RSI_BEAR_TF': 8, 'RSI_BEAR_HIGH': 57.46995465236236, 'RSI_BEAR_LOW': 24.005702805940228, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 15, 'ADX': 10, 'ADX_TF': 13, 'ADX_HIGH': 81.64531842257551, 'ADX_LOW': 21.6834731576957, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 0.3089068361825446, 'TAKE_PROFIT': 36.159684423955504}
# eos
eos_pr = {'RSI_BULL': 6, 'RSI_BULL_TF': 15, 'RSI_BULL_HIGH': 86.38380534611292, 
          'RSI_BULL_LOW': 47.82670374300257, 'RSI_BEAR': 14, 'RSI_BEAR_TF': 12, 'RSI_BEAR_HIGH': 58.382946812346546, 'RSI_BEAR_LOW': 16.168399447943912, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 15, 'ADX': 17, 'ADX_TF': 15, 'ADX_HIGH': 67.85830811681194, 'ADX_LOW': 50.129331043687664, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 6.498605150940965, 'TAKE_PROFIT': 20.52519819696282 }

eth_pr = {'RSI_BULL': 13, 'RSI_BULL_TF': 10, 'RSI_BULL_HIGH': 65.62609414161363, 'RSI_BULL_LOW': 53.52454178730807, 'RSI_BEAR': 18, 'RSI_BEAR_TF': 6, 'RSI_BEAR_HIGH': 34.38405675565822, 'RSI_BEAR_LOW': 25.03899435873855, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 8, 'ADX': 17, 'ADX_TF': 8, 'ADX_HIGH': 76.20346166290349, 'ADX_LOW': 35.04703818610004, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 2.51303150563804, 'TAKE_PROFIT': 32.410741740642116}

In [113]:
def run_strategy(data, params):
    # calculating
    rsi_bull = talib.RSI(data['close'], timeperiod=params['RSI_BULL'])
    rsi_bear = talib.RSI(data['close'], timeperiod=params['RSI_BEAR'])
    sma_slow = talib.SMA(data['close'], timeperiod=params['SML_SLOW'])
    sma_fast = talib.SMA(data['close'], timeperiod=params['SMA_FAST'])
    adx = talib.ADX(high=data['high'], low=data['low'], close=data['close'], timeperiod=params['ADX'])

    order_history = []
    entry_price = 0
    exit_price = 0
    dd = 0
    ru = 0
    dd_percent = 0
    ru_percent = 0

    sma_timeframe = params['SMA_TF']
    adx_timeframe = params['ADX_TF']
    rsi_bull_timeframe = params['RSI_BULL_TF']
    rsi_bear_timeframe = params['RSI_BEAR_TF']
    rsi_bull_high = params['RSI_BULL_HIGH']
    rsi_bull_low = params['RSI_BULL_LOW']
    rsi_bear_high = params['RSI_BEAR_HIGH']
    rsi_bear_low = params['RSI_BEAR_LOW']
    adx_high = params['ADX_HIGH']
    adx_low = params['ADX_LOW']
    bull_mod_high = params['BULL_MOD_HIGH']
    bull_mod_low = params['BULL_MOD_LOW']
    bear_mod_high = params['BEAR_MOD_HIGH']
    bear_mod_low = params['BEAR_MOD_LOW']
    stop_loss = params['STOP_LOSS']  # %
    take_profit = params['TAKE_PROFIT']  # %

    sma_slow_count = 0
    sma_fast_count = 0
    adx_count = 0
    rsi_bull_count = 0
    rsi_bear_count = 0

    in_position = False
    in_fake_position = False  # position if dont have stop loss/take profit
    sma_fast_value = 0
    sma_slow_value = 0
    adx_value = 0

    for index, row in data.iterrows():
        price = row['close']
        # SMA slow
        if sma_slow_count >= sma_timeframe:
            sma_slow_value = sma_slow[index]
            sma_slow_count = 0
        else:
            sma_slow_count += 1

        # SMA fast
        if sma_fast_count >= sma_timeframe:
            sma_fast_value = sma_fast[index]
            sma_fast_count = 0
        else:
            sma_fast_count += 1

        # ADX
        if adx_count >= adx_timeframe:
            adx_value = adx[index]
            adx_count = 0
        else:
            adx_count += 1

        # RSI bull
        rsi_bull_value = rsi_bull[index]

        # RSI bear
        rsi_bear_value = rsi_bear[index]

        is_bull_market = sma_fast_value > sma_slow_value
        is_bear_market = sma_fast_value < sma_slow_value

        # dd, ru
        if in_position:
            dd = min(dd, price)
            dd_percent = 100 * (dd / entry_price - 1)
            ru = max(ru, price)
            ru_percent = 100 * (ru / entry_price - 1)
            # stop/take profit
            if (1 - price / entry_price) > stop_loss / 100 or (price / entry_price - 1) > take_profit / 100:
                # print("Sell at: ", price)
                # print("------------")
                exit_price = price
                # save history
                order_history.append(
                    {"entry": entry_price, "exit": exit_price, "profit": 100 * (exit_price / entry_price - 1),
                     "draw_down": dd, "run_up": ru, "draw_down_percent": dd_percent, "run_up_percent": ru_percent})
                in_position = False
        else:
            dd = 0
            ru = 0
            dd_percent = 0
            ru_percent = 0

        if is_bull_market:
            if adx_value > adx_high:
                rsi_bull_high_adx = rsi_bull_high + bull_mod_high
            elif adx_value < adx_low:
                rsi_bull_low_adx = rsi_bull_low + bull_mod_low
            if not in_fake_position and not in_position and rsi_bull_value >= rsi_bull_high:
                # print("------------")
                # print("Buy at: ", price)
                in_position = True
                in_fake_position = True
                entry_price = price
                dd = ru = price
            elif in_position and rsi_bull_value <= rsi_bull_low:
                # print("Sell at: ", price)
                # print("------------")
                exit_price = price
                # save history
                order_history.append(
                    {"entry": entry_price, "exit": exit_price, "profit": 100 * (exit_price / entry_price - 1),
                     "draw_down": dd, "run_up": ru, "draw_down_percent": dd_percent, "run_up_percent": ru_percent})
                in_position = False
                in_fake_position = False
            elif rsi_bull_value <= rsi_bull_low:
                in_fake_position = False

        elif is_bear_market:
            if adx_value > adx_high:
                rsi_bear_high_adx = rsi_bear_high + bear_mod_high
            elif adx_value < adx_low:
                rsi_bear_low_adx = rsi_bear_low + bear_mod_low
            if not in_fake_position and not in_position and rsi_bear_value >= rsi_bear_high:
                # print("------------")
                # print("Buy at: ", price)
                in_position = True
                in_fake_position = True
                entry_price = price
                dd = ru = price
            elif in_position and rsi_bear_value <= rsi_bear_low:
                # print("Sell at: ", price)
                # print("------------")
                exit_price = price
                order_history.append(
                    {"entry": entry_price, "exit": exit_price, "profit": 100 * (exit_price / entry_price - 1),
                     "draw_down": dd, "run_up": ru, "draw_down_percent": dd_percent, "run_up_percent": ru_percent})
                in_position = False
                in_fake_position = False
            elif rsi_bear_value <= rsi_bear_low:
                in_fake_position = False
    return pd.DataFrame(order_history)

In [114]:
def performance_report(order_df):
    total_profit = order_df['profit'].sum()
    max_dd = order_df['draw_down_percent'].min()
    max_ru = order_df['run_up_percent'].max()
    num_win = order_df[order_df.profit >= 0].count().profit
    num_loss = order_df[order_df.profit < 0].count().profit
    total_win_profit = order_df[order_df.profit >= 0].sum().profit
    total_loss_profit = order_df[order_df.profit < 0].sum().profit
    ave_win = total_win_profit / num_win
    ave_loss = total_loss_profit / num_loss
    win_rate = num_win / (num_win + num_loss)
    start_price = order_df.iloc[0]['entry']
    end_price = order_df.iloc[-1]['exit']
    market = (end_price / start_price - 1) * 100
    start_balance = 100
    balance = start_balance * (1 + total_profit / 100)
    relative_profit = balance - start_balance
    relative_yearly_profit = relative_profit / 3
    sharpe = 10
    trades = order_df.shape[0] * 2

    return {'profit': float(total_profit), 'max_dd': float(max_dd), 'max_ru': float(max_ru),
            'num_win': int(num_win), 'num_loss': int(num_loss),
            'total_win_profit': int(total_win_profit),
            'total_loss_profit': int(total_loss_profit),
            'ave_win': float(ave_win), 'ave_loss': float(ave_loss),
            'win_rate': float(win_rate), 'balance': float(balance), 'startBalance': float(start_balance),
            'relativeYearlyProfit': float(relative_yearly_profit), 'market': float(market), 'sharpe': float(sharpe),
            'trades': int(trades), 'startPrice': start_price, 'endPrice': end_price, 'relativeProfit': relative_profit}


In [122]:
order_history_df = run_strategy(data_df, eth_pr)


In [123]:
order_history_df


,entry,exit,profit,draw_down,run_up,draw_down_percent,run_up_percent
0,411.35,409.85,-0.364653,409.15,412.50,-0.534824,0.279567
1,404.50,408.85,1.075402,404.50,410.20,0.000000,1.409147
2,367.60,363.35,-1.156148,363.35,368.70,-1.156148,0.299238
3,328.80,317.00,-3.588808,317.00,328.80,-3.588808,0.000000
4,280.05,281.10,0.374933,278.60,288.00,-0.517765,2.838779
...,...,...,...,...,...,...,...
321,447.10,450.70,0.805189,440.30,454.20,-1.520913,1.588012
322,459.55,457.75,-0.391688,453.75,477.10,-1.262104,3.818953
323,460.00,458.60,-0.304348,458.45,463.80,-0.336957,0.826087
324,468.40,462.95,-1.163535,462.95,468.40,-1.163535,0.000000


In [126]:
order_history_df.to_csv("result/ETHUSD-30m-results.csv")


In [127]:
performance_report(order_history_df)


{'profit': 298.60752898735086,
 'max_dd': -6.9158878504672945,
 'max_ru': 35.32513181019332,
 'num_win': 124,
 'num_loss': 202,
 'total_win_profit': 636,
 'total_loss_profit': -338,
 'ave_win': 5.136563820448364,
 'ave_loss': -1.6748830928131002,
 'win_rate': 0.3803680981595092,
 'balance': 398.60752898735086,
 'startBalance': 100.0,
 'relativeYearlyProfit': 99.53584299578363,
 'market': 15.06016774036707,
 'sharpe': 10.0,
 'trades': 652,
 'startPrice': 411.35,
 'endPrice': 473.3,
 'relativeProfit': 298.60752898735086}

In [119]:
# backtest
# Brute force
total_profit = 0
while True:
    params = {
        'RSI_BULL': random.randint(5, 20),
        'RSI_BULL_TF': random.randint(5, 20),
        'RSI_BULL_HIGH': random.uniform(60, 100),
        'RSI_BULL_LOW' : random.uniform(40, 60),
        'RSI_BEAR': random.randint(5, 20),
        'RSI_BEAR_TF': random.randint(5, 20),
        'RSI_BEAR_HIGH': random.uniform(30, 60),
        'RSI_BEAR_LOW' : random.uniform(5, 30),
        'SMA_FAST': 138,
        'SML_SLOW': 67,
        'SMA_TF': random.randint(5, 20),
        'ADX': random.randint(5, 20),
        'ADX_TF': random.randint(5, 20),
        'ADX_HIGH': random.uniform(60, 100),
        'ADX_LOW': random.uniform(5, 60),
        'BULL_MOD_HIGH': 5,
        'BULL_MOD_LOW' : -5,
        'BEAR_MOD_HIGH': 15,
        'BEAR_MOD_LOW' : -5,
        'STOP_LOSS': random.uniform(0.1, 10), # %,
        'TAKE_PROFIT': random.uniform(0.2, 50) # %
    }
    odf = run_strategy(data_df, params)
    pr  = performance_report(odf)
    if total_profit <= pr['profit']:
        total_profit = pr['profit']
        print("------------------------")
        print ("Best profit: ", total_profit)
        print ("Params: ", params)
        print("------------------------")
        

------------------------
Best profit:  173.96257501425333
Params:  {'RSI_BULL': 20, 'RSI_BULL_TF': 12, 'RSI_BULL_HIGH': 89.01635223454278, 'RSI_BULL_LOW': 43.64012721544347, 'RSI_BEAR': 9, 'RSI_BEAR_TF': 19, 'RSI_BEAR_HIGH': 37.66546102068891, 'RSI_BEAR_LOW': 20.175462213170107, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 15, 'ADX': 8, 'ADX_TF': 5, 'ADX_HIGH': 96.3273087180674, 'ADX_LOW': 56.74698310132674, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 9.203765978695568, 'TAKE_PROFIT': 25.323931565714258}
------------------------
------------------------
Best profit:  183.27420478881345
Params:  {'RSI_BULL': 7, 'RSI_BULL_TF': 5, 'RSI_BULL_HIGH': 90.56216470196611, 'RSI_BULL_LOW': 45.721051694100076, 'RSI_BEAR': 6, 'RSI_BEAR_TF': 15, 'RSI_BEAR_HIGH': 55.3328618473287, 'RSI_BEAR_LOW': 29.775936718441375, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 19, 'ADX': 5, 'ADX_TF': 12, 'ADX_HIGH': 88.76988386301744, 'ADX_LOW': 33.77852862222706, 'BULL_M

KeyboardInterrupt: 

In [41]:
# backtest
# GA
from pyeasyga import pyeasyga

# setup data
data = data_df

ga = pyeasyga.GeneticAlgorithm(data)        # initialise the GA with data

def create_individual(data):
    individual = [
        random.randint(5, 20),
        random.randint(5, 20),
        random.uniform(60, 100),
        random.uniform(40, 60),
        random.randint(5, 20),
        random.randint(5, 20),
        random.uniform(30, 60),
        random.uniform(5, 30),
        random.randint(70, 200),
        random.randint(5, 70),
        random.randint(5, 20),
        random.randint(2, 10),
        random.randint(5, 10),
        random.uniform(60, 100),
        random.uniform(5, 60),
        5,
        -5,
        15,
        -5,
        random.uniform(0.1, 10), # %,
        random.uniform(0.2, 50) # %
    ]
    return individual

ga.create_individual = create_individual

# define a fitness function
def fitness(individual, data):
#     print (individual)
    params = {
        'RSI_BULL': individual[0],
        'RSI_BULL_TF': individual[1],
        'RSI_BULL_HIGH': individual[2],
        'RSI_BULL_LOW' : individual[3],
        'RSI_BEAR': individual[4],
        'RSI_BEAR_TF': individual[5],
        'RSI_BEAR_HIGH': individual[6],
        'RSI_BEAR_LOW' : individual[7],
        'SMA_FAST': individual[8],
        'SML_SLOW': individual[9],
        'SMA_TF': individual[10],
        'ADX': individual[11],
        'ADX_TF': individual[12],
        'ADX_HIGH': individual[13],
        'ADX_LOW': individual[14],
        'BULL_MOD_HIGH': individual[15],
        'BULL_MOD_LOW' : individual[16],
        'BEAR_MOD_HIGH': individual[17],
        'BEAR_MOD_LOW' : individual[18],
        'STOP_LOSS': individual[19], # %,
        'TAKE_PROFIT': individual[20] # %
    }
#     params = {'RSI_BULL': 18, 'RSI_BULL_TF': 14, 'RSI_BULL_HIGH': 70.98853186912616, 'RSI_BULL_LOW': 46.045622334549414, 'RSI_BEAR': 16, 'RSI_BEAR_TF': 8, 'RSI_BEAR_HIGH': 57.46995465236236, 'RSI_BEAR_LOW': 24.005702805940228, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 15, 'ADX': 10, 'ADX_TF': 13, 'ADX_HIGH': 81.64531842257551, 'ADX_LOW': 21.6834731576957, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 0.3089068361825446, 'TAKE_PROFIT': 36.159684423955504}
#     params['STOP_LOSS'] = individual[19]
    odf = run_strategy(data, params)
    pr  = performance_report(odf)
    print ("----------------")
    print ("Profit: ",pr['total_profit'])
    return float(pr['total_profit'])

ga.fitness_function = fitness               # set the GA's fitness function
ga.run()                                    # run the GA
print (ga.best_individual())                 # print the GA's best solution

----------------
Profit:  -81.64430633671773
----------------
Profit:  -35.22827160322875
----------------
Profit:  29.129080627035655
----------------
Profit:  -78.21283539147771
----------------
Profit:  -53.31857336506793
----------------
Profit:  -28.520179042958414
----------------
Profit:  -115.7829102745516
----------------
Profit:  29.36168536657781
----------------
Profit:  7.370989789696456
----------------
Profit:  -76.13239967018566
----------------
Profit:  -37.760054487088816
----------------
Profit:  38.16031732021988
----------------
Profit:  -129.96276832737297
----------------
Profit:  -87.82845851726844
----------------
Profit:  -140.13498957633215
----------------
Profit:  -115.03080492523213
----------------
Profit:  -128.28484987248154
----------------
Profit:  -38.12983232030214
----------------
Profit:  -51.65655854062649
----------------
Profit:  37.39866335299159
----------------
Profit:  -72.25971212688529
----------------
Profit:  -36.86953010472265
--------

KeyboardInterrupt: 

In [33]:
pr

{'total_profit': -119.67190574024251,
 'max_dd': 23.12,
 'max_ru': 363.61,
 'num_win': 90,
 'num_loss': 202,
 'total_win_profit': 552.4039082928518,
 'total_loss_profit': -672.075814033094,
 'ave_win': 6.1378212032539095,
 'ave_loss': -3.3271079902628418,
 'win_rate': 0.3082191780821918}